In [1]:
import json
import requests
import psycopg2
import pandas as pd
import os

### add data to .bashrc- file
in command window in root /home/kasia open **nano .bashrc**
after that add write export XXXX=password f.exp. **DATABASE_NAME=string**

in python we **import os** and use **os.environ.get('DATABASE_NAME')** to get the data

# Homework
The practical exercise you can get started on and that we will review during the session next Monday:
    - Sign up to use this api:
https://calendarific.com/api-documentation
    - Get the attached json file with a list of countries and store it into your Jupyter folder
From Jupyter:
    -  load the list of countries from the json and for each country
get all holidays for year 2021 using the api
    - Output the holidays to a local json file or Postgres table, with following attributes for each holiday:
    country_name
    holiday_name
    holiday_date
    type_observance

In [98]:
# fitler out only country code and request informations for holiday
def collect_data_for_countries(countries_codes:list,year:int)->list:
    """ open json-file with countries codes
        download data for countries in json-file from calendarific API and append to list"""

    API_URL = 'https://calendarific.com/api/v2/holidays?&api_key={}&country={}&year={}'    
    holiday_responses = []
    token = os.environ.get('TOKEN_CALENDARIFIC')
    countries_codes = list(set([c['code'] for c in countries_codes]))
    for code in countries_codes: 
        response = requests.get(API_URL.format(token,code,year))
        holiday_responses.append(json.loads(response.text))
    return holiday_responses
          


def select_holiday_info(holiday_responses:list)-> dict:  
    """  
        select data from list of dict
        append dict info for every holiday to list
        take country name for last record in response
        collect data to dict
        dict FORMAT example
        {'Tunisia': 
                [{'holiday_name': 'New Year','holiday_date': '2021-01-01','type_observance': 'National holiday'},
                  {'holiday_name': 'Revolution and Youth Day','holiday_date': '2021-01-14','type_observance': 'National holiday'}
                ]
        }
        """

    holiday_in_country = {}
    for response in holiday_responses:
        holiday_list = []
        for holiday_info in response['response']['holidays']:
            info = {
                'holiday_name': holiday_info['name'],
                'holiday_date':holiday_info['date']['iso'],
                'type_observance':holiday_info['type'][0]
                }
            holiday_list.append(info)
        holiday_in_country[holiday_info['country']['name']] = holiday_list
    return holiday_in_country 
#holiday = json.dumps(holiday_in_country,indent=3)
#print(holiday)

#### add to table

def prepare_holiday_info_for_df(holiday_responses:dict)->list:
    """ 
        prepare list of tuples which will be added to postgres table
    """


    country_list = []
    hol_info = []
    for country in  holiday_responses:
        country_list.append((country,))
        for hol in holiday_responses[country]:
            hol_info.append((country,hol['holiday_name'],hol['holiday_date'],hol['type_observance']))
    return hol_info,country_list

def insert_holiday_info_to_table(postgres_insert_query:str,hol_info):
    """insert data to table which is choose in query"""
    cur.executemany(postgres_insert_query, hol_info)
    conn.commit()

def create_pandas_table(sql_query, database = conn):
    table = pd.read_sql_query(sql_query, database)
    return table

In [108]:
#with open('country_list.json','r') as country_file:
    #countries_codes = json.load(country_file)


#holiday_responses = collect_data_for_countries(countries_codes,2021) 

#with open('country_holiday.json','w') as country_holiday_file:
    #json.dump(holiday_responses,country_holiday_file)
    
with open('country_holiday.json','r') as country_holiday_file:
    holiday_responses = json.load(country_holiday_file)
    
holiday_in_country = select_holiday_info(holiday_responses)

#with open('selected_holiday_info.json','w') as country_holiday_file:
    #json.dump(holiday_in_country,country_holiday_file)
holiday = json.dumps(holiday_in_country,indent=2)
print(holiday)


{
  "Tunisia": [
    {
      "holiday_name": "New Year",
      "holiday_date": "2021-01-01",
      "type_observance": "National holiday"
    },
    {
      "holiday_name": "Revolution and Youth Day",
      "holiday_date": "2021-01-14",
      "type_observance": "National holiday"
    },
    {
      "holiday_name": "March Equinox",
      "holiday_date": "2021-03-20T10:37:30+01:00",
      "type_observance": "Season"
    },
    {
      "holiday_name": "Independence Day",
      "holiday_date": "2021-03-20",
      "type_observance": "National holiday"
    },
    {
      "holiday_name": "Martyrs' Day",
      "holiday_date": "2021-04-09",
      "type_observance": "National holiday"
    },
    {
      "holiday_name": "Labour Day",
      "holiday_date": "2021-05-01",
      "type_observance": "National holiday"
    },
    {
      "holiday_name": "Eid al-Fitr",
      "holiday_date": "2021-05-13",
      "type_observance": "National holiday"
    },
    {
      "holiday_name": "Eid al-Fitr Holiday",


In [99]:
#connection with freuenloop database 
conn = psycopg2.connect(host='localhost', port = 5432, database='frauenloop', user=os.environ.get('USERNAME_POSTGRES'), password=os.environ.get('USERNAME_PASSWORD'))
cur = conn.cursor()
conn.commit()

with open('selected_holiday_info.json','r') as country_holiday_file:
    holiday_responses = json.load(country_holiday_file)
hol_info,country_list = prepare_holiday_info_for_df(holiday_responses)
#insert_holiday_info_to_table(""" INSERT INTO holiday (country_name,holiday_name,holiday_date,type_observance) VALUES (%s,%s,%s,%s)""",hol_info)
#insert_holiday_info_to_table(""" INSERT INTO country (country_name) VALUES (%s)""",country_list)

#create_pandas_table('SELECT * FROM "country"')

In [101]:
country_list 

[('Tunisia',),
 ('Belgium',),
 ('Guinea-Bissau',),
 ('Iraq',),
 ('El Salvador',),
 ('Namibia',),
 ('Kuwait',),
 ('East Timor',),
 ('Nauru',),
 ('eSwatini',),
 ('Panama',),
 ('Lebanon',),
 ('Serbia',),
 ('Greenland',),
 ('Grenada',),
 ('South Africa',),
 ('New Caledonia',),
 ('Bangladesh',),
 ('Bulgaria',),
 ('Gambia',),
 ('Kazakhstan',),
 ('Albania',),
 ('Monaco',),
 ('Honduras',),
 ('Eritrea',),
 ('Reunion',),
 ('South Sudan',),
 ('Argentina',),
 ('Peru',),
 ('Cayman Islands',),
 ('Philippines',),
 ('Maldives',),
 ('Wallis and Futuna',),
 ('Czechia',),
 ('Venezuela',),
 ('North Macedonia',),
 ('Chile',),
 ('Turkey',),
 ('Costa Rica',),
 ('Colombia',),
 ('Laos',),
 ('Lithuania',),
 ('United States',),
 ('Cyprus',),
 ('Ecuador',),
 ('Somalia',),
 ('United Kingdom',),
 ('Spain',),
 ('Syria',),
 ('Burkina Faso',),
 ('Samoa',),
 ('Hungary',),
 ('Gabon',),
 ('Kiribati',),
 ('Greece',),
 ('British Virgin Islands',),
 ('Cuba',),
 ('Cameroon',),
 ('Denmark',),
 ('Saint Vincent and the Grenadin